In [20]:
# Install required libraries
!pip install PyPDF2

# Import required libraries
import PyPDF2
import itertools
import multiprocessing
import string
import time
from google.colab import files

# Upload the PDF file
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]
print(f"Uploaded PDF file: {pdf_filename}")

# ================================
# ✅ Function to check if the PDF can be opened with a given password
# ================================
def try_open_pdf(pdf_filename, password):
    try:
        with open(pdf_filename, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            if reader.is_encrypted:
                if reader.decrypt(password):
                    print(f"[+] Password found: {password}")
                    return True
                else:
                    return False
            else:
                print("[+] PDF is not password protected.")
                return True
    except Exception as e:
        print(f"Error: {e}")
        return False

# ================================
# ✅ Worker task to try a batch of passwords (moved to global scope)
# ================================
def worker_task(args):
    pdf_filename, passwords = args
    for password in passwords:
        if try_open_pdf(pdf_filename, password):
            return password
    return None

# ================================
# ✅ Brute-force with parallel processing
# ================================
def brute_force_parallel(pdf_filename, max_length, num_workers=4):
    characters = string.ascii_lowercase + string.digits
    password_found = None

    # Generate all possible password combinations
    def generate_passwords(length):
        for pwd in itertools.product(characters, repeat=length):
            yield ''.join(pwd)

    # Run brute-force for each length
    for length in range(1, max_length + 1):
        print(f"[*] Trying passwords of length: {length}")
        passwords = list(generate_passwords(length))

        # Split passwords into chunks for parallel processing
        chunk_size = max(1, len(passwords) // num_workers)
        chunks = [passwords[i:i + chunk_size] for i in range(0, len(passwords), chunk_size)]

        # Prepare arguments for worker_task
        task_args = [(pdf_filename, chunk) for chunk in chunks]

        # Use multiprocessing Pool to try passwords in parallel
        with multiprocessing.Pool(num_workers) as pool:
            results = pool.map(worker_task, task_args)

        # Check if password was found in any of the results
        for result in results:
            if result is not None:
                password_found = result
                break

        if password_found:
            print(f"[+] Password found: {password_found}")
            return password_found

    print("[-] Password not found.")
    return None

# ================================
# ✅ Save unlocked PDF if password is found
# ================================
def unlock_pdf(pdf_filename, password):
    with open(pdf_filename, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        if reader.decrypt(password):
            writer = PyPDF2.PdfWriter()
            for page_num in range(len(reader.pages)):
                writer.add_page(reader.pages[page_num])

            output_filename = f"unlocked_{pdf_filename}"
            with open(output_filename, 'wb') as output_file:
                writer.write(output_file)

            print(f"[+] Unlocked PDF saved as: {output_filename}")
            files.download(output_filename)

# ================================
# ✅ Run brute-force attack
# ================================
# Set max password length and number of workers
max_length = 3  # Increase length if needed
num_workers = multiprocessing.cpu_count()

# Start brute-force attack
start_time = time.time()
found_password = brute_force_parallel(pdf_filename, max_length, num_workers=num_workers)
end_time = time.time()

# Show time taken
print(f"[*] Time taken: {end_time - start_time:.2f} seconds")

# Unlock PDF if password is found
if found_password:
  print("password is:" + found_password)
  unlock_pdf(pdf_filename, found_password)


Saving pdf_protected (1)_protected.pdf to pdf_protected (1)_protected (1).pdf
Uploaded PDF file: pdf_protected (1)_protected (1).pdf
[*] Trying passwords of length: 1
[*] Trying passwords of length: 2
[*] Trying passwords of length: 3
[+] Password found: 123
[+] Password found: 123
[*] Time taken: 164.14 seconds
password is:123
[+] Unlocked PDF saved as: unlocked_pdf_protected (1)_protected (1).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>